# Broadcast Joins

# 1 Load Data

In [1]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

In [2]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [3]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))

In [4]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

## 1.2 Load Station Metadata

In [5]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# 2 Standard Joins

In [16]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

## 2.1 Original Execution Plan

In [17]:
df = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
df.explain(True)

== Parsed Logical Plan ==
Join Inner, ((usaf#87 = USAF#122) && (wban#88 = WBAN#123))
:- Project [year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_direction#92, substring(value#82, 64, 1) AS wind_direction_qual#93, substring(value#82, 65, 1) AS wind_observation#94, (cast(cast(substring(value#82, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#95, substring(value#82, 70, 1) AS wind_speed_qual#96, (cast(cast(substring(value#82, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
:  +- Project [value#82, 2003 AS year#84]
:     +- Relation[value#82] text
+- Relation[USAF#122,WBAN#123,STATION NAME#124,CTRY#125,STATE#126,ICAO#127,LAT#128,LON#129,ELEV(M)#130,BEGIN#131,END#132] csv

== Analyzed Logical Plan 

## 2.2 Explicit Broadcast Joins

In [13]:
df = weather.join(broadcast(stations), (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
df.explain(True)

== Parsed Logical Plan ==
Join Inner, ((usaf#87 = USAF#122) && (wban#88 = WBAN#123))
:- Project [year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_direction#92, substring(value#82, 64, 1) AS wind_direction_qual#93, substring(value#82, 65, 1) AS wind_observation#94, (cast(cast(substring(value#82, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#95, substring(value#82, 70, 1) AS wind_speed_qual#96, (cast(cast(substring(value#82, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
:  +- Project [value#82, 2003 AS year#84]
:     +- Relation[value#82] text
+- ResolvedHint (broadcast)
   +- Relation[USAF#122,WBAN#123,STATION NAME#124,CTRY#125,STATE#126,ICAO#127,LAT#128,LON#129,ELEV(M)#130,BEGIN#131,END#132]

## 2.3 Automatic Broadcast Joins

In [14]:
spark.conf.unset("spark.sql.autoBroadcastJoinThreshold")

In [15]:
df = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
df.explain(True)

== Parsed Logical Plan ==
Join Inner, ((usaf#87 = USAF#122) && (wban#88 = WBAN#123))
:- Project [year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_direction#92, substring(value#82, 64, 1) AS wind_direction_qual#93, substring(value#82, 65, 1) AS wind_observation#94, (cast(cast(substring(value#82, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#95, substring(value#82, 70, 1) AS wind_speed_qual#96, (cast(cast(substring(value#82, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
:  +- Project [value#82, 2003 AS year#84]
:     +- Relation[value#82] text
+- Relation[USAF#122,WBAN#123,STATION NAME#124,CTRY#125,STATE#126,ICAO#127,LAT#128,LON#129,ELEV(M)#130,BEGIN#131,END#132] csv

== Analyzed Logical Plan 